# Analitika podataka

In [ ]:
# !conda install matplotlib -y

Uspješno smo izlučili tekstualne podatke iz naših PDF dokumenata, sljedeći korak je analizirati dobivene podatke. Podatke preuzmite s Merlina - [link](https://moodle.srce.hr/2024-2025/mod/resource/view.php?id=4436168).

In [ ]:
# Učitavanje izlučenih podataka
import pandas as pd

podaci = pd.read_pickle("./DATASETS/ED4RE_2603_UTOR.pickle")

Pogledajmo osnovne informacije:

In [ ]:
podaci.info()

Naši podaci se sastoje od 14 stupaca koji uključuju naslov rada, afilijacije, DOI, časopis u kojem je rad objavljen te ključne riječi.



> **Zadatak 1**: Koliko različitih časopisa se pojavljuje u skupu podataka? Koliko se puta pojavljuje svaki od časopisa?

In [ ]:
# Zadatak 1
print("Ukupno različitih časopisa: ", len(podaci.Journal.unique()))

print("\nPojavnost pojedinog časopisa: ")
print(podaci["Journal"].value_counts())

Pogledajmo izlučene ključne riječi:

In [ ]:
print(podaci.Keywords)

## Analiza ključnih riječi

Neki od istraživačkih radova ne posjeduju ključne riječi ili iste nisu uspješno izlučene. Također, ključne riječi su zapisane u retcima na više različitih načina: lista, string, csv.

> **Zadatak 2**: Funkcijom normalizirajte zapise ključnih riječi u skupu podataka.

In [ ]:

# Zadatak 2
import re

keywords = []

for row in podaci[podaci.Keywords != "no_keywords"]["Keywords"]:

    # Omotajmo stringove u listu
    if isinstance(row, list):
        temp_0 = row
    else:
        temp_0 = [row]

    # Riješimo se string lista
    temp_1 = []
    for r in temp_0:
        r_temp = re.split("[,;(\\xa0)·(\n)]", r)

        temp_1.extend(r_temp)

    # Riješimo se praznih članova
    temp_2 = [r.strip().lower() for r in temp_1 if len(r.strip()) > 0] 
    # print(temp_2)

    keywords.append(temp_2)


In [ ]:
print(keywords)

### Frekvencija

Izračunajmo frekvenciju pojavljivanja pojedine ključne riječi u skupu podataka.

In [ ]:
import matplotlib.pyplot as plt

# Napravimo 1D listu kljućnih riječi
all_keywords = []
for sublist in keywords:
    for keyword in sublist:
        all_keywords.append(keyword)

# Izračunavanje frekvencije
keyword_counts = {}
for keyword in all_keywords:
    if keyword in keyword_counts:
        keyword_counts[keyword] += 1
    else:
        keyword_counts[keyword] = 1

# Sortirajmo ključne riječi prema frekvenciji
sorted_keywords = sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)
top_keywords = sorted_keywords[:10]

# Frekvencija ključnih riječi
print("10 najfrekventnijih ključnih riječi:")
for keyword, count in top_keywords:
    print(f"{keyword}: {count}")

# Vizualizacija
plt.figure(figsize=(10, 6))
plt.bar([kw[0] for kw in top_keywords], [kw[1] for kw in top_keywords], color='skyblue')
plt.xlabel('Ključne riječi')
plt.ylabel('Frekvencija')
plt.title('10 najfrekventnijih ključnih riječi:')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


### Prosječna duljina ključnih riječi

> **Zadatak 3**: Izračunajte prosječnu duljinu ključnih riječi.
>
> **Zadatak 4**: Izračunajte medijan duljine ključnih riječi.

In [ ]:
# Duljina svake ključne riječi
keyword_lengths = [len(keyword) for keyword in all_keywords]

# Zadatak 3
# Prosječna duljina
average_length = sum(keyword_lengths) / len(keyword_lengths)

print(f"Prosječna duljina ključnih riječi: {average_length:.2f}")
# Zadatak 4
print(f"Medijan duljine ključnih riječi: {sorted(keyword_lengths)[int(len(keyword_lengths)/2)]}")

# Distribucija duljina ključnih riječi
plt.hist(keyword_lengths, bins=range(1, max(keyword_lengths) + 2), color='skyblue', edgecolor='black')
plt.xlabel('Duljina ključne riječi')
plt.ylabel('Frekvencija')
plt.title('Distribucija duljine ključnih riječi')
plt.show()


### Mreža ključnih riječi

> **Zadatak 5**: Pridružite pojedinom bridu mreže broj pojavljivanja brida. - Težina/Frekvencija

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations

G = nx.Graph()


for group in keywords:

    G.add_nodes_from(group)

    G.add_edges_from(combinations(group, 2))

# Vizualizacija
plt.figure(figsize=(12, 8))
pos = nx.random_layout(G) 
nx.draw_networkx_nodes(G, pos, node_color='skyblue', node_size=[5 * G.degree(node)**2 for node in G.nodes()])
nx.draw_networkx_edges(G, pos, edge_color='grey')
nx.draw_networkx_labels(G, pos, font_size=8, font_color='black')
plt.title("Mreža ključnih riječi", fontsize=14)
plt.axis('off')
plt.show()

## Analiza sadržaja

### NLTK

Za analizu tekstualnog sadržaja istraživačkih radova koristiti ćemo [NLTK](https://www.nltk.org/) biblioteku. NLTK biblioteka vodeća je platforma za izradu Python programa za rad s podacima u obliku prirodnog jezika.

In [ ]:
# !conda install nltk -y

Primjer tokenizacije:

In [ ]:
import nltk
# nltk.download('punkt_tab')

primjer_rečenice = podaci["Content"].iloc[1].split(".")[0] + "."

print("Rečenica             : ", primjer_rečenice)
print("Tokenizirana rečenica: ", nltk.word_tokenize(primjer_rečenice))

### Frekvencija bigrama

Proučimo sadržaj kroz pojavljivanje parova riječi - bigrame.

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt


text = " ".join(podaci.Content.tolist())

# Tokenizacija u riječi
# nltk.download('punkt')
tokens = word_tokenize(text.lower())

# Stvaranje bigrama
bigrams = list(ngrams(tokens, 2))

# Izračunavanje frekvencije bigrama
bigram_freq = Counter(bigrams)


N = 5
print(f"Top {N} bigrama po frekventnosti:")
for bigram, freq in bigram_freq.most_common(N):
    print(f"{bigram}: {freq}")

# Vizualizacija
top_bigrams = bigram_freq.most_common(N)
bigrams_labels = [' '.join(bigram) for bigram, _ in top_bigrams]
frequencies = [freq for _, freq in top_bigrams]

plt.barh(bigrams_labels, frequencies)
plt.xlabel('Frekvencija')
plt.ylabel('Bigrami')
plt.title(f'Top {N} bigrama po frekvenciji')
plt.show()


Primjećujemo da naši rezultati nisu pretjerano informativni, najfrekventniji bigrami sastoje se od čestih i neinformativnih riječi, primjerice: *of, the, in, the*.

>> Stop words are the words in a stop list (or stoplist or negative dictionary) which are filtered out (i.e. stopped) before or after processing of natural language data (text) because they are deemed insignificant.
>>
>> https://en.wikipedia.org/wiki/Stop_word

> **Pitanje**: Koje su riječi stop-riječi u hrvatskom jeziku?


Izbacimo stop riječi:

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

# Izbacivanje stop-riječi
stop_words = set(stopwords.words('english'))
# stop_words.update(['et', 'al', 'n', 'r'])  # Dodatne stop riječi
filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]

# Stvaranje bigrama
bigrams = list(ngrams(filtered_tokens, 2))

# Izračunavanje frekvencije bigrama
bigram_freq = Counter(bigrams)


N = 5
print(f"Top {N} bigrama po frekventnosti:")
for bigram, freq in bigram_freq.most_common(N):
    print(f"{bigram}: {freq}")

# Vizualizacija
top_bigrams = bigram_freq.most_common(N)
bigrams_labels = [' '.join(bigram) for bigram, _ in top_bigrams]
frequencies = [freq for _, freq in top_bigrams]

plt.barh(bigrams_labels, frequencies)
plt.xlabel('Frekvencija')
plt.ylabel('Bigrami')
plt.title(f'Top {N} bigrama po frekvenciji')
plt.show()


### WordCloud

In [ ]:
# !conda install wordcloud -y

In [ ]:
from wordcloud import WordCloud

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
words = [word.lower() for word in text.split() if word.isalnum() and word.lower() not in stop_words]

filtered_text = " ".join(words)

wordcloud = WordCloud(background_color="White").generate(filtered_text)

# Vizualizacija
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud", fontsize=16)
plt.show()


### TF-IDF

In [ ]:
# !conda install scikit-learn -y

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# NLTK stop riječi
stop_words = set(stopwords.words('english'))

# Dodavanje vlasitith stop riječi
stop_words.update(["et", "al"])

# TF-IDF izračun
vectorizer = TfidfVectorizer(stop_words=list(stop_words))
tfidf_matrix = vectorizer.fit_transform(podaci.Content.tolist())

# Izračun TF-IDF "score"
feature_names = vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.sum(axis=0).A1

tfidf_results = sorted(zip(feature_names, tfidf_scores), key=lambda x: x[1], reverse=True)

# Prikaz top pojmova
N = 20
print(f"Top {N} pojmova s obzirom na TF-IDF:")
for word, score in tfidf_results[:N]:
    print(f"{word}: \t\t {score:.4f}")

In [ ]:
import numpy as np
feature_names = vectorizer.get_feature_names_out()

N = 10  
top_tfidf_terms = []

for i in range(tfidf_matrix.shape[0]):
    # Sve TF-IDF vrijednosti za pojedini dokument
    tfidf_scores = tfidf_matrix[i].toarray().flatten()
    
    # Top N TF-IDF vrijednosti za pojedini dokument
    top_indices = np.argsort(tfidf_scores)[::-1][:N]
    
    # Dohvaćanje pojmova
    top_terms = [(feature_names[idx], tfidf_scores[idx]) for idx in top_indices]
    
    top_tfidf_terms.append(top_terms)


for i, top_terms in enumerate(top_tfidf_terms):
    print(f"Dokument {i+1} - Top 10 TF-IDF Pojmova:")
    for term, score in top_terms:
        print(f"{term}: {score:.4f}")
    print()